### From a File


In [ ]:
from main import create_taxonomy

filename = "cais_data_expanded.csv"
brand_terms = [
    "cais",
    "cais group",
    "glas",
    "glas funds",
    "halo",
    "halo investing",
    "icapital",
    "icapital network",
]
website_subject = "Alternate Investing Platform"

taxonomy, df, samples = create_taxonomy(
    filename,
    text_column="keyword",
    search_volume_column="search_volume",
    min_df=5,
    brand_terms=brand_terms,
)

df.to_csv("cais_data_taxonomy.csv", index=False)

print("\n".join(taxonomy))

In [ ]:
from main import create_taxonomy

filename = "HM Raw Data.csv"
website_subject = "Houston Methodist Hospital"

brand_terms = [
    "luke",
    "lukes",
    "md anderson",
    "anderson",
    "hca",
    "stlukes",
    "memorial hermann",
    "hermann",
    "herman",
    "houston methodist",
    "methodist",
    "st joseph",
    "joseph",
]

taxonomy, df, samples = create_taxonomy(
    filename,
    website_subject=website_subject,
    text_column="keyword",
    search_volume_column="search_volume",
    min_cluster_size=10,
    min_samples=3,
    limit_queries=3,
    ngram_range=(1, 5),
    min_df=10,
    brand_terms=brand_terms,
)


df.to_csv("HM_Raw_Data_ngram_taxonomy2.csv", index=False)
print("\n".join(taxonomy))

### From a GSC Account

In [1]:
from main import create_taxonomy

brand_terms = ["ledgeloungers", "ledge"]
property = "sc-domain:ledgeloungers.com"
website_subject = "Ledge Lounger offers luxury pool & outdoor furniture designed to create perfect spaces for outdoor entertaining and relaxation."

taxonomy, df, samples = create_taxonomy(
    property,
    website_subject=website_subject,
    days=30,
    ngram_range=(1, 5),
    cross_encoded=True,
    min_df=5,
    brand_terms=brand_terms,
    limit_queries_per_page=5,
)


df.to_csv("ledgeloungers_taxonomy2.csv", index=False)

df.head()

c:\Users\jroak\anaconda3\envs\taxonomyml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-21 08:59:41.047 | INFO     | lib.searchconsole:load_gsc_account_data:51 - Creating dataframe...
2023-07-21 08:59:59.075 | INFO     | lib.searchconsole:load_gsc_account_data:58 - Dataframe created.
2023-07-21 08:59:59.410 | INFO     | main:create_taxonomy:215 - Got Data. Dataframe shape: (2419, 7)
2023-07-21 08:59:59.412 | INFO     | main:create_taxonomy:217 - Filtering Query Data.
2023-07-21 08:59:59.500 | INFO     | main:score_and_filter_df:110 - Got ngram frequency. Dataframe shape: (384, 2)
2023-07-21 08:59:59.555 | INFO     | main:score_and_filter_df:113 - Merged Ngrams. Dataframe shape: (321, 4)
2023-07-21 09:00:01.524 | INFO     | main:score_and_filter_df:143 - Final score and filter length: 321
2023-07-21 09:00:

   ngram_size    query  merged_frequency  frequency  search_volume     score
0           1  lounger          1.000000   1.000000       1.000000  1.000000
1           1     pool          0.154122   0.690695       0.836386  0.495254
2           1     hole          0.014337   0.013106       0.560620  0.287479
3           1    table          0.111111   0.328965       0.423304  0.267208
4           1  outdoor          0.301075   0.250328       0.218655  0.259865


2023-07-21 09:00:10.897 | INFO     | main:create_taxonomy:234 - Reviewing OpenAI's work.
2023-07-21 09:00:17.030 | INFO     | main:create_taxonomy:254 - Getting structure.
2023-07-21 09:00:17.031 | INFO     | main:create_taxonomy:258 - Adding categories.
2023-07-21 09:00:17.033 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 09:00:17.965 | INFO     | lib.clustering:fit_pairwise_crossencoded:479 - Getting embeddings.
2023-07-21 09:00:17.967 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 09:00:17.969 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2
Batches: 100%|██████████| 22/22 [00:02<00:00,  7.50it/s]
2023-07-21 09:00:21.442 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 09:00:21.443 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2
2023-07-21 09:00:22.073 | INFO     | lib.clustering:fit_

PermissionError: [Errno 13] Permission denied: 'ledgeloungers_taxonomy.csv'

In [2]:
df.to_csv("ledgeloungers_taxonomy2.csv", index=False)

In [9]:
import pandas as pd
from lib.clustering import ClusterTopics

df = pd.read_csv("ledgeloungers_taxonomy.csv")

df.head()

queries = list(set(df["query"].tolist()))
queries = [queries[idx] for idx in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]]
taxonomies = list(set(df["taxonomy"].tolist()))

cluster_model = ClusterTopics(cluster_categories=taxonomies)
cluster_model.corpus = queries
cluster_model.embeddings = cluster_model.get_embeddings(queries)


2023-07-21 07:28:42.942 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 07:28:42.944 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2


2023-07-21 07:28:43.519 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 07:28:43.521 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

category_embeddings = cluster_model.get_embeddings(taxonomies)
query_embeddings = cluster_model.embeddings

query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
category_embeddings = category_embeddings / np.linalg.norm(category_embeddings, axis=1, keepdims=True)


cosine_similarity_matrix = cosine_similarity(query_embeddings, category_embeddings)


2023-07-21 07:28:49.580 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 07:28:49.582 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2


In [ ]:
idxs = 10
top_n = 5
import settings
from tqdm import tqdm
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/qnli-electra-base")
cross_encoder_pairs = []


for idx, query in enumerate(queries):
    #print("#" * 50)
    #print(f"Query: {query}")

    #print("\nCosine Similarity:")
    top_categories = np.argsort(cosine_similarity_matrix[idx])[-top_n:][::-1]
    pairs = []
    for i in top_categories:
        #print(taxonomies[i], cosine_similarity_matrix[idx][i])
        pairs.append((query, taxonomies[i]))
    
    cross_encoder_pairs.append(pairs)
    #print()


cross_encoder_similarity = np.array([
            cross_encoder.predict(pairs).flatten()
            for pairs in cross_encoder_pairs
        ])

for idx, query in enumerate(queries):
    print("#" * 50)
    print(f"Query: {query}")

    print("\nCross-encoder Similarity:")
    
    # Test cross_encoder_similarity scores for this query.  If all scores are high and similar, then the query is ambiguous.  If on score is considerably lower than the others, then the query is not ambiguous.
    


    top_category= np.argmin(cross_encoder_similarity[idx])
    print(cross_encoder_pairs[idx][top_category][1], cross_encoder_similarity[idx][top_category])
    print()



In [15]:
import settings
from tqdm import tqdm
from sentence_transformers import CrossEncoder
import numpy as np

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")

query = "in the pool chairs"

options = """Chairs Pool Chairs
Pool Furniture Chairs
Pool Furniture Loungers
Loungers Pool Loungers
Pool Furniture Accessories""".split("\n")

options = [o.lower() for o in options]

cross_encoder_pairs = [(query, option) for option in options]

cross_encoder_similarity = cross_encoder.predict(cross_encoder_pairs).flatten()

print("Query:", query)
print("Options:")
for idx, pair in enumerate(cross_encoder_pairs):
    print(pair[1], cross_encoder_similarity[idx])

print()
print(f"Best: {cross_encoder_pairs[np.argmax(cross_encoder_similarity)][1]}, {cross_encoder_similarity[np.argmax(cross_encoder_similarity)]}")


Query: in the pool chairs
Options:
chairs pool chairs 7.287129
pool furniture chairs 7.0691895
pool furniture loungers 3.595707
loungers pool loungers -2.3364563
pool furniture accessories 2.3251443

Best: chairs pool chairs, 7.287128925323486


In [1]:
from main import add_categories
import pandas as pd 

df = pd.read_csv("ledgeloungers_taxonomy2.csv")

structure = list(set(df.taxonomy.tolist()))

# Random sample of 10 df rows
df = df.sample(20)

df = add_categories(structure, df, 
                    cross_encoded=True, 
                    percentile_threshold=50,
                    std_dev_threshold= 0.1)




c:\Users\jroak\anaconda3\envs\taxonomyml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-21 10:12:44.159 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 10:12:44.160 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2
2023-07-21 10:12:45.330 | INFO     | lib.clustering:fit_pairwise_crossencoded:479 - Getting embeddings.
2023-07-21 10:12:45.331 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 10:12:45.333 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2
2023-07-21 10:12:45.903 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 10:12:45.904 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: a

In [12]:
import pandas as pd
from main import clean_provided_dataframe, score_and_filter_df

df_raw = pd.read_csv("out_barry.csv")
df = df_raw.copy()
df['query'] = df.title.copy()
df['search_volume'] = df.ga_sessions.copy()

df = clean_provided_dataframe(df, None, None)

df = score_and_filter_df(df, min_df=5, min_length=3000)

df.head()


2023-07-24 16:21:37.219 | INFO     | main:score_and_filter_df:111 - Got ngram frequency. Dataframe shape: (7697, 2)
2023-07-24 16:21:49.689 | INFO     | main:score_and_filter_df:114 - Merged Ngrams. Dataframe shape: (7004, 4)
Filtering by knee:   2%|▏         | 1/51 [00:00<00:01, 38.46it/s]
2023-07-24 16:30:03.586 | INFO     | main:score_and_filter_df:166 - Filtered Knee (sensitivity=1005). Dataframe shape: (3000, 6)


,ngram_size,query,merged_frequency,frequency,search_volume,score
0,1,google,1.000000,1.000000,1.000000,1.000000
1,2,google says,0.578231,0.048684,0.108810,0.343521
2,2,google search,0.321429,0.067480,0.309968,0.315698
3,1,seo,0.465986,0.049218,0.063091,0.264539
4,1,yahoo,0.385204,0.053717,0.001497,0.193350


In [13]:
df.to_csv("out_barry_ngrams4.csv", index=False)

In [22]:
from lib.api import get_openai_response_chat
import settings
from tqdm.auto import tqdm

df_entities = df_raw.copy().head(20)

entities_found = set()

result_dict = {}

for i, row in tqdm(df_entities.iterrows(), total=len(df_entities)):
    
    topic = row["title"]
    existing_entities = "\n".join(entities_found)

    prompt = f"""As an expert at SEO-subject detection in text content,
    You will be given a topic and a list of existing subjects. The topics are content found on a popular SEO news site called SERoundtable.com. Your job to find key subjects related to SEO and online marketing found in the topic. 
    For example:
      - Topic: google news suggested for you - to find sources, Subjects: Google News.
      - Topic: google removes related search operator, Subjects: Google, Search Operators.
      - Topic: google search central live indonesia event, Subjects: Google Search Central.
      - Topic: bing chat gets visual search, Subjects: Bing Chat, Visual Search.
    To keep things consistent, if the subject exists in the existing subjects, please use the subject from that list. Also, only use the singular form of subjects. Your output should be a comma-separated list of subjects in proper case. 

    Topic: {topic}

    Existing subjects: {existing_entities}

    Subjects: """

    explanation = get_openai_response_chat(
        prompt,
        model=settings.OPENAI_QUALITY_MODEL,
        system_message="You are an expert subject detection in text content.",
    )

    entities = [e.strip() for e in explanation.split(",")]
    entities_found.update(entities)

    result_dict[topic] = entities





  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:21<00:00,  1.07s/it]


In [15]:
from lib.api import get_openai_response_chat
import settings
from tqdm.auto import tqdm

ngrams = """Redirects
404 Errors
Ad Units
Advertisers
Affiliate Links
Alexa
Google Update
Algorithm Update
Algorithms
Alphabet
Alt Text
Amit Singhal
AMP
Anchor Text
Android
AOL
API
Apple
Article
Ask Jeeves
Ask.com
Attribution
Author
Authority
Authorship
Bard
Barry Schwartz
Bert
Bing
Bing Ads
Bing Chat
Bing Search
Bing Search Results
Bing Shopping
Bing Webmaster Tools
Bingbot
Birthday
Black Hat
Blogger
Bots
Breadcrumbs
Broad Match
Business Listings
Business Profile
Buying Links
Cache
Campaigns
Canonical
Carousel
ChatGPT
Google Chrome
Apple Safari
Google Hummingbird Update
Click Fraud
Cloaking
Conference
Content
Core Updates
Core Web Vitals
Covid 19
Crawling
CSS
CTR
Danny Sullivan
Desktop Search Results
Disallow
Disavow File
Disavow Links
DMOZ
DNS
Dogpile
Domains
Domain Name
Doodle
DuckDuckGo
Duplicate Content
Dynamic Search Ads
Easter Egg
Ebay
Emojis
Eric Schmidt
Evergreen Googlebot
Experiments
External Links
Facebook
Facebook Search
Favicon
Featured Snippet
Firefox
Follow
Forums
Gambling
Gary Illyes
Gmail
Google Ads
Google Ads Editor
Google Ads Performance
Google Ads Search
Google AdSense
Google Adsense Account
Google AdSense Ads
Google AdSense Publishers
Google AdWords
Google AdWords Account
Google AdWords Ads
Google Adwords API
Google AdWords Campaigns
Google AdWords Editor
Google Alerts
Google AMP
Google Analytics
Google Android
Google Answers
Google API
Google App
Google App Indexing
Google Assistant
Google Bard
Google Blog Search
Google My Business
Google Business App
Google Business Insights
Google Business Listings
Google My Business App
Google My Business Insights
Google My Business Listings
Google My Business Profiles
Google Business Profiles
Google Cache
Google Caffeine
Google Changes
Google Checkout
Google Dance
Google Day
Google Desktop
Google Disavow Tool
Google Discover
Google Doodle
Google Earth
Google Fiber
Google Finance
Google Glass
Google Helpful Content Update
Google Home
Google Home Page
Google Images
Google Instant
Google Keyword Tool
Knowledge Graph
Google Lens
Google Local
Google Local Business Center
Google Local Maps
Google Local Pack
Google Local Panel
Google Local Algorithm
Google Local Algorithm Update
Google Logo
Google Maps
Google Maps App
Google Maps Business
Google Maps Search
Google Medic Update
Google Merchant Center
Google Mobile
Google Mobile Index
Google News
Google News Publishers
Google Office
Google Page Experience Update
Google PageRank Update
Google Pages
Google Pagespeed Insights
Google Panda Update
Google Partner
Google Patent
Google Penguin Update
Google People Search
Google Places
Google Play
Google Plus
Google Plus Local
Google Product Reviews Update
Google Product Search
Google Rankings
Google Reader
Google Real Time
Google Results
Google Reviews Update
Google Rich Results
Google Sandbox
Google Search
Google Search Analytics
Google Search App
Google Search Box
Google Search Bug
Google Search Console
Google Search Console API
Google Search Console Data
Google Search Console Reports
Google Search Results
Google Search Testing
Google Self Driving Car
Google Shopping
Google Shopping Ads
Google Sitemaps
Google Spam
Google Stories
Google Street View
Google Suggest
Google Tests
Google Toolbar
Google Translate
Google Trends
Google Videos
Google Video Search
Google Voice Search
Google Webmaster Central
Google Webmaster Conference
Google Webmaster Team
Google Webmaster Report
Google Webmaster Tools
Google Webmaster
Google.com
Googlebot
Googleguy
Googleplex
Googlers
Guest Blogging
Hacked
Headers
Home Page
Hosting
Hreflang
HTML5
Image Carousel
Google Image Search
Image Search
Images
Indexing
IndexNow
Internal Links
Internet Explorer
iOS
IP Address
iPhone
JavaScript
John Mueller
Keyword Density
Keyword Planner
Keywords
Knowledge Panel
Knowledte Panel
Landing Pages
Links
Link Building
Google Link Spam Update
Google Local Business
Local Knowledge Panel
Local Listings
Local Pack
Local Results
Google Local Results
Google Local Search
Google Local Service Ads
Location
Logo
Low Quality
AI
Machine Learning
Malware
Manual Actions
Marissa Mayer
Markup
Martin Splitt
Matt Cutts
Meta
Meta Descriptions
Meta Tag
Microsoft
Microsoft AdCenter
Microsoft Advertising
Microsoft Bing
Microsoft Live Search
Microsoft Search
Mobile Friendly
Mobile Friendly Algorithm
Mobile Index
Mobile Indexing
Mobile Results
Mountain View
Moz
Mozilla
MSN
MSN AdCenter
MSN Search Update
MSN Search
MSNbot
Navigation Bar
Negative SEO
Nofollow
Noindex
Organic Results
Organic Search
Overture
Page Experience
PageRank
Paid Links
PDFs
Penalty
Performance Max
Performance Report
Pubcon
Quality Content
Quality Raters Guidelines
Quality Score
Rand Fishkin
RankBrain
Ranking Changes
Ranking Factor
Ranking Signal
Rating
Recipe
Referrer
Results Page
Reviews
Rich Snippets
Robots.txt
RSS
Sandbox
Scams
Schema
Search Ads
Search Box
Search Community
Search Community Honors
Search Engines
Search Engine Friendly
Search Engine Land
Search Marketing
Search Engine Roundtable
Search Engine Watch
Search Generative Experience
Search History
Search Quality
Search Rankings
Search Results
Search Suggestions
Selling Links
SEM
SEMPO
SEO
SEO Industry
SEOMoz
Sergey Brin
Service Ads
Shopify
Shopping Ads
Signals
Site Command
Sitelinks
Sitemaps
SMX
Snippets
Social Media
Spam
Spam Algorithm
Google Speed Update
Google Backlink Update
SSL
Status Code
Structured Data
Sundar Pichai
Tags
Targeting
Text Ads
Theory
Title Tags
Titles
TLDs
Topic
Traffic
Trending Searches
Tweets
Twitter
Unnatural Links
URLs
URL Inspection Tool
User Agent
Videos
Web Search
Webmaster Central
Webmaster Guidelines
Webmaster Report
Webmasters
WebmasterWorld
Webspam
Widget
Wikipedia
Word Count
Wordpress
XML Sitemaps
Yahoo
Yahoo Directory
Yahoo Publisher Network
Yahoo Search
Yahoo Search Marketing
Yahoo Search Update
Yahoo Site Explorer
Yandex
Yelp
YouTube
YouTube Videos
Keyword Research
Google Florida Update
Google Toolbar PageRank Update
Google Fred Update
Google Farmer Update
Google Maccabees Update
Exact Match Domains
Google Phantom Update"""

PROMPT = f"""As an expert at SEO, I need your help to review a list of topics and organize them into relevant categories.

Here are some examples you can use to understand the expected output:
  Ad Units, Paid
  Google Analytics, Analytics & Tools
  Carousel, Search Results
  Google Panda Update, Algorithm Update
  Google Caffeine,	Algorithm
  Google Penguin Update, Algorithm
  Google Hummingbird Update, Algorithm
  Google Finance, Google Products
  Google Maps, Google Products
  Google My Business, Local SEO
  Firefox, Browsers
  Google Chrome, Browsers
  ChatGPT, Machine Learning
  Google Lens, Google Products
  Cloaking, SEO Tactics
  Canonical,	Technical SEO
  Google Search Console, Analytics & Tools


Topics for Review:
{ngrams}

You MUST follow the following format for your response:
<original ngram: text-only>, <category: singular proper-case text-only>
<original ngram: text-only>, <category: singular proper-case text-only>
. . .

Please read the examples very carefully to try to unsderstand why each category was assigned before you begin.

Begin!"""


explanation = get_openai_response_chat(
        PROMPT,
        model=settings.OPENAI_LARGE_MODEL,
        system_message="You are an expert at SEO History.",
    )

print(explanation)

Redirects, Technical SEO
404 Errors, Technical SEO
Ad Units, Paid Advertising
Advertisers, Paid Advertising
Affiliate Links, SEO Tactics
Alexa, Analytics & Tools
Google Update, Algorithm Update
Algorithm Update, Algorithm Update
Algorithms, Algorithm
Alphabet, Company
Alt Text, On-Page SEO
Amit Singhal, Google Employees
AMP, Mobile SEO
Anchor Text, On-Page SEO
Android, Mobile SEO
AOL, Search Engines
API, Technical SEO
Apple, Company
Article, Content
Ask Jeeves, Search Engines
Ask.com, Search Engines
Attribution, SEO Tactics
Author, On-Page SEO
Authority, SEO Tactics
Authorship, On-Page SEO
Bard, Google Employees
Barry Schwartz, SEO Community
Bert, Algorithm Update
Bing, Search Engines
Bing Ads, Paid Advertising
Bing Chat, Search Engines
Bing Search, Search Engines
Bing Search Results, Search Results
Bing Shopping, Search Engines
Bing Webmaster Tools, Analytics & Tools
Bingbot, Bots
Birthday, Google Events
Black Hat, SEO Tactics
Blogger, Content Management
Bots, Bots
Breadcrumbs, On-Pag

In [7]:
import pandas as pd
# import StringIO
from io import StringIO

# read in the csv file from string `explanation` using StringIO

df = pd.read_csv(StringIO(explanation), names=['ngram', 'category'])

df.to_csv('categories_gpt.csv')


In [1]:
730 * 0.521

380.33000000000004